In [1]:
using ITensors, ITensorMPS
using LinearAlgebra
using SparseArrays
using Plots
using JLD2

In [22]:
include("src/main.jl")
include("src/observables.jl")
include("src/states.jl")
include("src/doubled_measurements.jl")
include("src/ED.jl")

ED_full (generic function with 1 method)

In [23]:
function MPS_update_data(ρ::MPS, observables::Vector{Symbol}, data::Dict{Symbol,Vector{Float64}}, t::Int; ref=false)
    if :Ic in observables
        data[:Ic][t] = Ic2(ρ)
    end
    if :SR in observables
        data[:SR][t] = SR(ρ)
    end
    if :κEA in observables
        data[:κEA][t] = κEA(ρ; ref=ref)
    end
    if :κ2 in observables
        data[:κ2][t] = κ2(ρ; ref=ref)
    end
    if :maxlinkdim in observables
        data[:maxlinkdim][t] = maxlinkdim(ρ)
    end
    return data
end

MPS_update_data (generic function with 1 method)

In [24]:
function circuit(L::Int, T::Int, λ::Float64, δ::Float64, q::Float64; observables=Symbol[], cutoff=1E-8, maxdim=200)
    ρ, sites = ghz(L; ref=true)

    data = Dict([s => zeros(Float64, 2T+2) for s in observables])
    data = MPS_update_data(ρ, observables, data, 1; ref=true)
    data = MPS_update_data(ρ, observables, data, 2; ref=true)

    ZZ_ms = zeros(Bool, T, L-1)
    X_ms = zeros(Bool, T, L)
    ZZ_vals = zeros(Float64, T, L-1)
    X_vals = zeros(Float64, T, L)

    λzz = δ*(1-λ)
    λx = δ*λ

    Xn = decoherence_layer(sites, PauliX, q, 1:L, L)
    ZZn = decoherence_layer(sites, kron(PauliZ, PauliZ), q, 1:L-1, L)
    bell_state = bell(sites)


    for t in 1:T
        ρ, X_ms[t,:], X_vals[t,:] = doubled_measure(ρ, PauliX, λx, 1:L; ref=true)
        ρ /= inner(bell_state, ρ)
        truncate!(ρ; cutoff=cutoff, maxdim=maxdim)

        ρ = apply(Xn, ρ)
        ρ /= inner(bell_state, ρ)
        truncate!(ρ; cutoff=cutoff, maxdim=maxdim)

        data = MPS_update_data(ρ, observables, data, 2t+1; ref=true)

        ρ, ZZ_ms[t,:], ZZ_vals[t,:] = doubled_measure(ρ, kron(PauliZ,PauliZ), λzz, 1:L-1; ref=true)
        ρ /= inner(bell_state, ρ)
        truncate!(ρ; cutoff=cutoff, maxdim=maxdim)

        ρ = apply(ZZn, ρ)
        ρ /= inner(bell_state, ρ)
        truncate!(ρ; cutoff=cutoff, maxdim=maxdim)

        data = MPS_update_data(ρ, observables, data, 2t+2; ref=true)
    end

    return ρ, data, (X_ms, ZZ_ms), (X_vals, ZZ_vals)
end

circuit (generic function with 1 method)

In [68]:
L = 6
T = 2L
λ = 0.2
δ = 0.7
q = 0.1
ρ, data, ms, vals = circuit(L,T,λ,δ,q; cutoff=1E-16, maxdim=400)

(MPS
[1] ((dim=2|id=480|"Qubit,Site,n=1"), (dim=2|id=599|"Link,l=1"))
[2] ((dim=2|id=943|"Qubit,Site,n=2"), (dim=4|id=234|"Link,l=2"), (dim=2|id=599|"Link,l=1"))
[3] ((dim=2|id=278|"Qubit,Site,n=3"), (dim=8|id=31|"Link,l=3"), (dim=4|id=234|"Link,l=2"))
[4] ((dim=2|id=78|"Qubit,Site,n=4"), (dim=16|id=944|"Link,l=4"), (dim=8|id=31|"Link,l=3"))
[5] ((dim=2|id=435|"Qubit,Site,n=5"), (dim=32|id=651|"Link,l=5"), (dim=16|id=944|"Link,l=4"))
[6] ((dim=2|id=331|"Qubit,Site,n=6"), (dim=64|id=217|"Link,l=6"), (dim=32|id=651|"Link,l=5"))
[7] ((dim=2|id=831|"Qubit,Site,n=7"), (dim=86|id=54|"Link,l=7"), (dim=64|id=217|"Link,l=6"))
[8] ((dim=2|id=617|"Qubit,Site,n=8"), (dim=64|id=127|"Link,l=8"), (dim=86|id=54|"Link,l=7"))
[9] ((dim=2|id=505|"Qubit,Site,n=9"), (dim=32|id=551|"Link,l=9"), (dim=64|id=127|"Link,l=8"))
[10] ((dim=2|id=508|"Qubit,Site,n=10"), (dim=16|id=23|"Link,l=10"), (dim=32|id=551|"Link,l=9"))
[11] ((dim=2|id=450|"Qubit,Site,n=11"), (dim=8|id=157|"Link,l=11"), (dim=16|id=23|"Link,l=10

In [69]:
L = 6
T = 2L
λ = 0.2
δ = 0.7
q = 0.1
ρ2, vals2, data = ED_full(L,T,λ,δ,q, ms[1], ms[2])

(sparse([1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  119, 120, 121, 122, 123, 124, 125, 126, 127, 128], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  128, 128, 128, 128, 128, 128, 128, 128, 128, 128], [0.0019176981428444702, -3.492363515472949e-5, 2.5580676871382873e-5, 1.928731465399848e-6, -2.5018742300519056e-5, 3.039486390014794e-7, -3.301240458559384e-5, -7.205351830040005e-6, -0.0005220689290259519, 3.6968046531219255e-6  …  3.6968046531219255e-6, -0.0005220689290259519, -7.205351830040005e-6, -3.301240458559384e-5, 3.039486390014794e-7, -2.5018742300519056e-5, 1.928731465399848e-6, 2.5580676871382873e-5, -3.492363515472949e-5, 0.0019176981428444702], 128, 128), ([0.0 0.0 … 0.0 0.0; 0.24883583500640438 -0.1036606451019502 … 0.02748069479964425 -0.07741764202088293; … ; -0.043985452345954454 -0.04510590526204593 … 0.07135770202665209 0.16346065588478845; 0.07075162265115681 -0.0388726461122686 … -0.059661455248702894 0.34129728731332365], [0.5917345473982598 0.17601370862607382 … 0.26694828061411024 0.

In [73]:
tol = 1E-7
all(abs.(vals[1] - vals2[1]) .< tol) && all(abs.(vals[2] - vals2[2]) .< tol)

true

In [67]:
all(abs.(to_matrix(ρ) .- ρ2) .< 1E-8)

true